In [2]:
import gzip
import json
import pickle
import urllib
import data
from data import load
import urllib.request
from tqdm import tqdm

prs = set()

def to_api_url(url: str):
    return url[:url.rindex("#")]\
        .replace("github.com/", "api.github.com/repos/")\
        .replace("/pull/", "/pulls/")

def to_issue_comments_url(api_url: str):
    return api_url.replace("pulls", "issues") + "/comments"

def to_comments_url(api_url: str):
    return api_url + "/comments"

# i = 0
# for l in tqdm(load(), total=25323640, desc="Records", unit='record'):
#     #print(l)
#     i += 1
#     if l["URL"] is not None:
#         prs.add(to_api_url(l["URL"]))
#         if '#discussion' not in (l["URL"] or "#discussion"):
#             print("Found: ", l)
#             break
# 
# pickle.dump(prs, gzip.open(data.prs_file, 'wb+', 9))

prs = pickle.load(gzip.open(data.prs_file, 'rb', 9))

print(len(prs), "PRs")

def get_data(url: str):
    api_url = to_api_url(url)
    api_data = json.loads(urllib.request.urlopen(api_url))
    
    closed = api_data['closed']
    merged = api_data['merged']


3863509 PRs
